In [98]:
import pandas as pd
import dask.dataframe as dd
import numpy as np


## Project 

In [1]:
import pandas as pd
import numpy as np

In [48]:
df = pd.read_csv('New_Dataset.csv', nrows = 300000)

In [50]:
def change_data_types(df):

    num_cols_to_convert = df.shape[1] - 5

    cols_to_convert = df.columns[:num_cols_to_convert]

    dtype_dict = {col: np.int8 for col in cols_to_convert}

    df[cols_to_convert] = df[cols_to_convert].astype(dtype_dict)
change_data_types(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Columns: 5134 entries, city_Aaronsburg to price
dtypes: float64(5), int8(5129)
memory usage: 1.4 GB


In [40]:
df['price'] = df['price'].astype('float32')

In [32]:
np.isinf(df["price"]).sum()

0

### Binning

In [72]:
percentile_33 = df['price'].quantile(0.33)
percentile_67 = df['price'].quantile(0.67)

In [73]:
price_list = df['price'].tolist()

In [74]:
bins_list = []
for i in price_list:
    if i <= percentile_33:
        bins_list.append("Low")
    elif i > percentile_33 and i <= percentile_67:
        bins_list.append("Middle")
    else:
        bins_list.append("High")
print(bins_list)

['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Middle', 'Middle', 'High', 'Middle', 'Middle', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Middle', 'Middle', 'Low', 'High', 'Middle', 'Middle', 'Low', 'Low', 'Low', 'Low', 'High', 'Middle', 'Low', 'High', 'Low', 'Low', 'High', 'Middle', 'Middle', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'High', 'Low', 'Middle', 'Low', 'Low', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Low', 'Middle', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'High', 'Low', 'Low', 'Middle', 'Middle', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'High', 'High', 'High', 'Low', 'Low', 'Low', 'Low', 

In [76]:
binned_label = pd.Series(bins_list)

In [96]:
print(binned_label[binned_label == "Low"].count() *100 / len(binned_label))
print(binned_label[binned_label == "Middle"].count() *100 / len(binned_label))
print(binned_label[binned_label == "High"].count() *100 / len(binned_label))

33.02
33.995333333333335
32.98466666666667


### Test-Train split

In [51]:
# split the data into train and test
features = df.drop(columns=['price'])
label = df["price"]

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
X_train , X_test , y_train , y_test = train_test_split(features, label, test_size=0.2, random_state=42)

### Logistic Regression

In [87]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

In [77]:
y_bins = binned_label

In [78]:
X_train , X_test , y_bins_train , y_bins_test = train_test_split(features, y_bins, test_size=0.2, random_state=42)

In [89]:
logistic_model = OneVsRestClassifier(LogisticRegression(solver='liblinear',  max_iter=10000))
logistic_model.fit(X_train, y_bins_train)

OneVsRestClassifier(estimator=LogisticRegression(max_iter=10000,
                                                 solver='liblinear'))

In [92]:
y_pred = logistic_model.predict(X_test)

In [93]:
#printing scores
print("Accuracy: ", accuracy_score(y_bins_test, y_pred))
print("Precision: ", precision_score(y_bins_test, y_pred, average='micro'))
print("Recall: ", recall_score(y_bins_test, y_pred, average='micro'))
print("F1 Score: ", f1_score(y_bins_test, y_pred, average='micro'))
print("Confusion Matrix: \n", classification_report(y_bins_test, y_pred))

Accuracy:  0.7107
Precision:  0.7107
Recall:  0.7107
F1 Score:  0.7107
Confusion Matrix: 
               precision    recall  f1-score   support

        High       0.75      0.79      0.77     19745
         Low       0.73      0.77      0.75     19692
      Middle       0.64      0.58      0.61     20563

    accuracy                           0.71     60000
   macro avg       0.71      0.71      0.71     60000
weighted avg       0.71      0.71      0.71     60000



### Linear Regression

In [55]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [56]:
model = LinearRegression()

In [57]:
model.fit(X_train, y_train)

LinearRegression()

In [58]:
y_pred = model.predict(X_test)

In [55]:
print(y_test.to_numpy())    

[ 80000. 415000. 199000. ...  58000. 159900. 190000.]


In [56]:
print(y_pred)

[  2048. 495616.  60416. ...  10496. 354048. 409600.]


In [59]:
r2 = r2_score(y_test, y_pred)


In [60]:
print(f"R-squared: {r2:.4f}")

R-squared: -40914423683918584.0000


### Trees 

#### CART

In [61]:
from sklearn.tree import DecisionTreeRegressor

In [62]:
cart_tree = DecisionTreeRegressor()  

In [63]:
cart_tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [64]:
y_pred = cart_tree.predict(X_test)

In [65]:
r2_trees = r2_score(y_test, y_pred)

In [66]:
print(f"R-squared: {r2_trees:.4f}")

R-squared: 0.7082


#### Random Forest

In [67]:
from sklearn.ensemble import RandomForestRegressor

In [68]:
random_forest = RandomForestRegressor(n_estimators=10, random_state=42)
random_forest.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=42)

In [69]:
y_pred = random_forest.predict(X_test)

In [70]:
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2:.4f}")

R-squared: 0.8041
